1. 데이터 수집

In [ ]:
import pandas as pd
from datetime import datetime
import os, os.path, json, requests
import time 

In [ ]:
api_key = 'RGAPI-2a2b64dd-3238-41e9-813e-2dc22de5f8d8'
target_player = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" +'변신하다' +'?api_key=' + api_key
r = requests.get(target_player)
r.json()

In [ ]:
tier_url = "https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/" + r.json()['id'] +'?api_key=' + api_key
r2  = requests.get(tier_url)
r2.json()

In [ ]:
master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(master)         # 마스터 데이터 호출
league_df = pd.DataFrame(r.json())

league_df.reset_index(inplace=True)   #수집한 마스터 데이터 index정리
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T #d ict구조로 되어 있는 entries컬럼 풀어주기
league_df = pd.concat([league_df, league_entries_df], axis=1) # 열끼리 결합

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.info()
league_df.to_csv('마스터데이터.csv',index=False,encoding = 'cp949')# 중간저장

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9588 entries, 0 to 9587
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          9588 non-null   object
 1   summonerId    9588 non-null   object
 2   summonerName  9588 non-null   object
 3   leaguePoints  9588 non-null   object
 4   wins          9588 non-null   object
 5   losses        9588 non-null   object
 6   veteran       9588 non-null   object
 7   inactive      9588 non-null   object
 8   freshBlood    9588 non-null   object
 9   hotStreak     9588 non-null   object
dtypes: object(10)
memory usage: 749.2+ KB


In [ ]:
league_df

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,MASTER,A840AmL2CvaoeK1QDidJKrJmQpwyAvCwrADcCZpn7vh-3Sc,qewrttt,156,154,120,False,False,False,True
1,MASTER,DpWV-9QWZEIkgfwVVhy941NbLmAreOy8FKmxpvpeMovtZsg,오 뉴,106,139,123,False,False,False,True
2,MASTER,zwQF4aZgJrEbp9VT6zRbhz_GWXKtN-GSuSpmdEwRl3Bpxqs,go ward me carry,110,129,115,False,False,True,False
3,MASTER,Jm_qvfK0Sp86EcKfJuKRmi6JMdcaKu5p8DxVGQVCYs7Dfg,럼 카,56,75,58,False,False,False,False
4,MASTER,y5GakRxvDrE8QGEReuptW37sutHEokV9Zgahsc786m4ZDzI,aband0ned dog,121,64,53,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
9583,MASTER,9g30pzoM4gzx5-ER1b7m2xBeICgEt3SWwBUE2wjx7YMIT_...,사람답게는 하자,22,212,204,False,False,True,False
9584,MASTER,-pH07bOit75nQTMDZA78i_F_oNTi-SXRpJ8kPYsLINMkMM...,싫어 니가 와,227,198,194,False,False,False,True
9585,MASTER,T7jpwavqjp67QjEXDgYufBsN7cRj_FqJ7vdmnGWQX5UADA,땡국2,1,192,183,False,False,True,False
9586,MASTER,ooqAw3l0xOrKis6px-SvdFlDSYIRbKAJVyRBwGKleIS9gw,앙 김우기,98,331,303,True,False,False,False


In [ ]:
for i in range(len(league_df)):
    try:
        target = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(target)
        
        while r.status_code == 429:
            time.sleep(5)
            target = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(target)
            
        account_id = r.json()['accountId']
        league_df.iloc[i, -1] = account_id
    
    except:
        pass

In [ ]:
len(league_df)

9584

2. 데이터 전처리